In [8]:
import os
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Normalize


In [9]:
class MyCNN(nn.Module): 
    # def __init__(self) -> None:
    #     super(MyCNN, self).__init__()
    #     self.stack = nn.Sequential(
    #         nn.Conv2d(3, 6, 5),
    #         nn.MaxPool2d(2, 2),
    #         nn.Conv2d(6, 16, 5),
    #         nn.MaxPool2d(2, 2),
    #         nn.Flatten(),
    #         nn.Linear(400, 128),
    #         nn.ReLU(),
    #         nn.Linear(128, 32),
    #         nn.ReLU(),
    #         nn.Linear(32, 10)
    #     )
        
    # def forward(self, x):
    #     return self.stack(x)
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 150)
        self.fc2 = nn.Linear(150, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
model = MyCNN()

In [13]:
input = torch.rand(3, 5, 5)
print(nn.MaxPool2d(F.relu(nn.Conv2d(3, 6, 5)(input))))


MaxPool2d(
  kernel_size=tensor([[[0.4040]],
  
          [[0.1349]],
  
          [[0.0000]],
  
          [[0.2061]],
  
          [[0.1280]],
  
          [[0.0000]]], grad_fn=<ReluBackward0>), stride=tensor([[[0.4040]],
  
          [[0.1349]],
  
          [[0.0000]],
  
          [[0.2061]],
  
          [[0.1280]],
  
          [[0.0000]]], grad_fn=<ReluBackward0>), padding=0, dilation=1, ceil_mode=False
)


In [3]:
learning_rate = 0.01
batch_size = 64
epochs = 10

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [4]:
tf = transforms.Compose([ToTensor(), Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=tf,
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=tf,
    # target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
)

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))

Files already downloaded and verified
Files already downloaded and verified


In [5]:
def train(dl: DataLoader, model: MyCNN, loss: nn.CrossEntropyLoss, opt: torch.optim.SGD):
    model.train()
    for x, y in dl:
        # x = x.to(device)
        # y = y.to(device)
        p = model(x)
        # print(p)
        # print(y)
        l = loss(p, y)
        l.backward()
        opt.step()
        opt.zero_grad()

In [6]:
def test(dl: DataLoader, model: MyCNN, loss: nn.CrossEntropyLoss):
    model.eval()
    tl, c = 0, 0
    sz = len(dl.dataset)
    b = len(dl)
    
    with torch.no_grad():
        for x, y in dl:
            # x = x.to(device)
            # y = y.to(device)
            p = model(x)
            tl += loss(p, y).item()
            c += (p.argmax(1) == y).type(torch.float).sum().item()
            
    tl /= b
    c /= sz
    
    print(tl)
    print(c)

In [7]:
for i in range(epochs):
    print(f"Epoch {i}")
    train(train_dataloader, model, criterion, optimizer)
    test(test_dataloader, model, criterion)

Epoch 0


PicklingError: Can't pickle <function <lambda> at 0x17a672de0>: attribute lookup <lambda> on __main__ failed